In [ ]:
# Generate the Metadata Bank

In [ ]:
import os
import re
import time
import torch
import pickle
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from pcdet.models import build_network, load_data_to_gpu
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils

In [ ]:
# Configs and checkpoints for all branches
branches = [['cfgs/waymo_models/centerpoint_dyn_pillar020_1x.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_pillar020_1x.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_pillar024_1x.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_pillar024_1x.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_pillar028_1x.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_pillar028_1x.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_pillar032_1x.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_pillar032_1x.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_pillar036_1x.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_pillar036_1x.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_pillar040_1x.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_pillar040_1x.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_voxel0075.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_voxel0075.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_voxel0100.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_voxel0100.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_voxel0125.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_voxel0125.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_voxel0150.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_voxel0150.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_voxel0175.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_voxel0175.pth'],
            ['cfgs/waymo_models/centerpoint_dyn_voxel0200.yaml',
                '../output/waymo_checkpoints/centerpoint_dyn_voxel0200.pth']]

baselines = [['cfgs/waymo_models/centerpoint_pillar_1x.yaml',
                '../output/waymo_baselines/centerpoint_pillar_1x.pth'],
            ['cfgs/waymo_models/centerpoint_without_resnet.yaml',
                '../output/waymo_baselines/centerpoint_without_resnet.pth'],
            ['cfgs/waymo_models/second.yaml',
                '../output/waymo_baselines/second.pth'],
            ['cfgs/waymo_models/PartA2.yaml',
                '../output/waymo_baselines/PartA2.pth'],
            ['cfgs/waymo_models/pointpillar_1x.yaml',
                '../output/waymo_baselines/pointpillar_1x.pth'],
            ['cfgs/waymo_models/pv_rcnn.yaml',
                '../output/waymo_baselines/pv_rcnn.pth']]

In [ ]:
# Generate look up table for Content Agnostic Scheduler
# Accuracy (mAP) of each branch on the test set
# Average Latency of each branch on the test set

# Get accuracy profiling results of all branches
# Level1 mAP of 3 classes: Vehicle, pedestrian, cyclist
acc = [0.708, 0.7075, 0.7048, 0.6948, 0.694, 0.6809, 0.7399, 0.7168, 0.6912, 0.6588, 0.6321, 0.5964]

# Get latency profiling results of all branches 
lat_orin = np.load('../output/waymo_results/latency_profiling/val/latency_profiling_val.npy', allow_pickle=True)

lat = []
for i, res in enumerate(lat_orin):
    e2e = (round(np.float64(lat_orin[i][1]), 2))
    lat.append(e2e)
print(lat)

look_up_content_ag = []
for i in range(len(lat)):
    look_up_content_ag.append([i, lat[i], acc[i]])
print(look_up_content_ag)
np.save('look_up_content_ag_orin', look_up_content_ag)

In [ ]:
import re
# Oracle controller
branches = ['centerpoint_dyn_pillar020_1x',
            'centerpoint_dyn_pillar024_1x',
            'centerpoint_dyn_pillar028_1x',
            'centerpoint_dyn_pillar032_1x',
            'centerpoint_dyn_pillar036_1x',
            'centerpoint_dyn_pillar040_1x',
            'centerpoint_dyn_voxel0075',
            'centerpoint_dyn_voxel0100',
            'centerpoint_dyn_voxel0125',
            'centerpoint_dyn_voxel0150',
            'centerpoint_dyn_voxel0175',
            'centerpoint_dyn_voxel0200']

data_root = '../output/waymo_results'

# latency profiles
print('Loading branch latency for test samples...')
lat_dir = os.path.join(data_root, 'latency_profiling/test')
branch_lats = []
for b in branches:
    lat_path = os.path.join(lat_dir, b + '_lat.pkl')
    lat = pickle.load(open(lat_path, 'rb'))
    branch_lats.append(np.array(lat))
branch_lats = np.stack(branch_lats, axis=-1)    # (#samples, #branches)

# accuracy profiles
print('Loading branch accuracy for test samples...')
acc_dir = os.path.join(data_root, 'accuracy_profiling/test')
branch_accs = []
for b in branches:
    acc_path = os.path.join(acc_dir, b + '.txt')
    with open(acc_path, "r") as file:
        content = file.read()
    pattern = r'\b\d+\.\d+\b'
    result = [[float(match) for match in re.findall(pattern, line)] for line in content.split("{")[1:]]
    indices = [0, 4, 12]
    acc = [sum(item[index] for index in indices) / len(indices) for item in result]
    branch_accs.append(np.array(acc))
branch_accs = np.stack(branch_accs, axis=-1)    # (#samples, #branches)

# load detection profiles
print('Loading branch detection results for test samples...')
branch_profiles = []
for b in branches:
    det_path = os.path.join(lat_dir, b + '_det.pkl')
    det = pickle.load(open(det_path, 'rb'))
    branch_profiles.append(det)
branch_profiles = np.stack(branch_profiles, axis=-1)    # (#samples, #branches)

In [ ]:
# print(branch_profiles)
print(branch_profiles.shape)
print(len(branch_profiles))
print(len(branch_profiles[0]))
# print(branch_profiles[0])

In [ ]:
# print(branch_accs)
print(branch_accs.shape)
print(len(branch_accs))
print(len(branch_accs[0]))
print(branch_accs[1])

In [ ]:
print(branch_lats)
print(branch_lats.shape)
print(len(branch_lats))
print(len(branch_lats[0]))
print(branch_lats[1])

In [ ]:
# latency budgets
board = 'orin'
slo_list = (100, 125, 150, 175, 200, 225, 250, 300, 400, 500)
# slo_list = (1000, 2000)
schd_overhead = 0

# SWITCH BETWEEN LATENCY PREDICTORS AND THRESHOLDS
print('Running virtual branch scheduling...')
det_results, lat_results = dict(), dict()
for slo in slo_list:
    det_results[slo] = []
    lat_results[slo] = []

# k is the number of frames
for k in range(len(branch_lats)):
    if (k + 1) % 500 == 0:
        print(f'{(k + 1):04d} done!')

    # for each frame, load the lat and acc of 20 branches
    lat, acc = branch_lats[k], branch_accs[k]

    for slo in slo_list:
        # Filter valid branches according to the latency slo
        valid_branches = np.nonzero(lat + schd_overhead <= slo)[0]
        # If there is no valid branch, choose the fastest branch
        if len(valid_branches) == 0:
            valid_branches = [lat.argmin()]
        # Select the branch come with the highest acc
        # If multiple branches have the same highest acc, choose the fastest branch
        sorted_indices = np.lexsort((lat[valid_branches], -acc[valid_branches]))
        b = valid_branches[sorted_indices[0]]
        lat_results[slo].append(lat[b])

        # load detection profile
        det = branch_profiles[k][b]
        det_results[slo].append(det)

print('Saving detection results...')
out_dir = '../output/waymo_results/oracle'
os.makedirs(out_dir, exist_ok=True)
for slo in slo_list:
    out_path = os.path.join(out_dir, f'{board}_slo{slo:d}_oracle.pkl')
    with open(out_path, 'wb') as f:
        pickle.dump(det_results[slo], f)

    lat = np.array(lat_results[slo]).mean()
    print(f"Latency [SLO: {slo:d}]: {lat:.2f}s")

In [ ]:
!export

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import pickle
from pathlib import Path
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils

config = 'cfgs/waymo_models/centerpoint_dyn_pillar020_1x.yaml'
# Read the config file
cfg_from_yaml_file(config, cfg)
cfg.TAG = Path(config).stem
cfg.EXP_GROUP_PATH = '/'.join(config.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'
np.random.seed(1024)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
dist_test = False
total_gpus = 1
batch_size = 1
workers = 4

# Create logger
logger = common_utils.create_logger()

# Build the dataloader
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=dist_test, workers=workers, logger=logger, training=False)
dataset = test_loader.dataset
class_names = dataset.class_names

# slo_list = (100, 125, 150, 175, 200, 225, 250, 300, 400, 500)
slo_list = (300, 400)
out_dir = '../output/waymo_results/oracle'
for slo in tqdm(slo_list):
    det_path = os.path.join(out_dir, f'orin_slo{slo:d}_oracle.pkl')
    final_output_dir = '../output/waymo_results/oracle/eval'
    os.makedirs(final_output_dir, exist_ok=True)
    # Read the detection results
    print('================', det_path, '=====================')
    det_annos = pickle.load(open(det_path, 'rb'))
    ret_dict = {}
    result_str, result_dict = dataset.evaluation(
            det_annos, class_names,
            eval_metric=cfg.MODEL.POST_PROCESSING.EVAL_METRIC,
            output_path=final_output_dir
        )

    ret_dict.update(result_dict)
    print(ret_dict)

In [1]:
# Profile on the validation set to generate LUR
import os
import numpy as np
from tqdm import tqdm
import pickle
from pathlib import Path
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils

config = 'cfgs/waymo_models/centerpoint_dyn_pillar020_1x.yaml'
# Read the config file
cfg_from_yaml_file(config, cfg)
cfg.TAG = Path(config).stem
cfg.EXP_GROUP_PATH = '/'.join(config.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'
np.random.seed(1024)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
dist_test = False
total_gpus = 1
batch_size = 1
workers = 4

# Create logger
logger = common_utils.create_logger()

# Build the dataloader
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=dist_test, workers=workers, logger=logger, training=False)
dataset = test_loader.dataset
class_names = dataset.class_names

branches = ['centerpoint_dyn_pillar020_1x',
            'centerpoint_dyn_pillar024_1x',
            'centerpoint_dyn_pillar028_1x',
            'centerpoint_dyn_pillar032_1x',
            'centerpoint_dyn_pillar036_1x',
            'centerpoint_dyn_pillar040_1x',
            'centerpoint_dyn_voxel0075',
            'centerpoint_dyn_voxel0100',
            'centerpoint_dyn_voxel0125',
            'centerpoint_dyn_voxel0150',
            'centerpoint_dyn_voxel0175',
            'centerpoint_dyn_voxel0200']

out_dir = '../output/waymo_results/latency_profiling/val'
for b in tqdm(branches[5:]):
    det_path = os.path.join(out_dir, b + '_det.pkl')
    final_output_dir = '../output/waymo_results/oracle/eval'
    os.makedirs(final_output_dir, exist_ok=True)
    # Read the detection results
    print('================', det_path, '=====================')
    det_annos = pickle.load(open(det_path, 'rb'))
    ret_dict = {}
    result_str, result_dict = dataset.evaluation(
            det_annos, class_names,
            eval_metric=cfg.MODEL.POST_PROCESSING.EVAL_METRIC,
            output_path=final_output_dir
        )

    ret_dict.update(result_dict)
    print(ret_dict)

/home/pengcheng/anaconda3/envs/openpcdet/lib/python3.8/site-packages/tqdm-4.66.1-py3.8.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-29 17:05:17,196   INFO  Loading Waymo dataset
2023-11-29 17:05:18,555   INFO  Total skipped info 0
2023-11-29 17:05:18,558   INFO  Total samples for Waymo dataset: 31895
  0%|          | 0/7 [00:00<?, ?it/s]

================ ../output/waymo_results/latency_profiling/val/centerpoint_dyn_pillar040_1x_det.pkl =====================


2023-11-29 17:05:21.342497: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Start the waymo evaluation...
Number: (pd, 6397649) VS. (gt, 1252669)
Level 1: 1082789, Level2: 169880)
Instructions for updating:
Use `self.session()` or `self.cached_session()` instead.


2023-11-29 17:05:29.023008: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-11-29 17:05:29.027905: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2023-11-29 17:05:29.028480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2023-11-29 17:05:29.028509: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-29 17:05:29.028541: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas

...

{'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_1/AP': 0.7135742, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_1/APH': 0.7075942, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_2/AP': 0.63604623, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_2/APH': 0.6306536, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_1/AP': 0.6445781, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_1/APH': 0.5809129, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_2/AP': 0.586747, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_2/APH': 0.52800894, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_1/AP': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_1/APH': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_2/AP': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_2/APH': 0.0, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_1/AP': 0.75969875, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_1/APH': 0.74564815, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_2/AP': 0.7156555, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_2/APH': 0.7024135}
================ ../output/waymo_results/latency_profiling/val/centerpoint_dyn_voxel0175_det.pkl =====================
Start the waymo evaluation...
Number: (pd, 7292340) VS. (gt, 1252669)
Level 1: 1082789, Level

2023-11-29 20:09:31.082076: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2023-11-29 20:09:31.082765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2023-11-29 20:09:31.082978: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-29 20:09:31.082992: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-29 20:09:31.083016: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuf

{'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_1/AP': 0.7054409, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_1/APH': 0.69943863, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_2/AP': 0.62767863, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_2/APH': 0.6222742, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_1/AP': 0.5825966, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_1/APH': 0.5188255, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_2/AP': 0.52810854, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_2/APH': 0.46969596, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_1/AP': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_1/APH': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_2/AP': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_2/APH': 0.0, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_1/AP': 0.74435914, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_1/APH': 0.73026484, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_2/AP': 0.70098865, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_2/APH': 0.6877111}
================ ../output/waymo_results/latency_profiling/val/centerpoint_dyn_voxel0200_det.pkl =====================
Start the waymo evaluation...
Number: (pd, 7165886) VS. (gt, 1252669)
Level 1: 1082789, L

2023-11-29 20:56:08.661684: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2023-11-29 20:56:08.662319: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2023-11-29 20:56:08.662571: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-29 20:56:08.662584: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-29 20:56:08.662592: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuf

{'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_1/AP': 0.6929585, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_1/APH': 0.6865042, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_2/AP': 0.6158777, 'OBJECT_TYPE_TYPE_VEHICLE_LEVEL_2/APH': 0.61008006, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_1/AP': 0.5220828, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_1/APH': 0.46007866, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_2/AP': 0.46842054, 'OBJECT_TYPE_TYPE_PEDESTRIAN_LEVEL_2/APH': 0.41245663, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_1/AP': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_1/APH': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_2/AP': 0.0, 'OBJECT_TYPE_TYPE_SIGN_LEVEL_2/APH': 0.0, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_1/AP': 0.7048927, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_1/APH': 0.69059384, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_2/AP': 0.6638565, 'OBJECT_TYPE_TYPE_CYCLIST_LEVEL_2/APH': 0.650386}
